### Prepare Unity Catalog AI

 - Initializes the Unity Catalog Function client
 - Implements a utility function to register a Unity Catalog Function

In [1]:
from typing import Callable
from unitycatalog.ai.core.client import UnitycatalogFunctionClient
from unitycatalog.client import ApiClient, Configuration


CLIENT = UnitycatalogFunctionClient(
    api_client=ApiClient(
        configuration=Configuration(host="http://localhost:8080/api/2.1/unity-catalog")
    )
)
CATALOG = "unity"
SCHEMA = "mcp"


def register(func: Callable[..., any]) -> None:
    CLIENT.create_python_function(
        catalog=CATALOG,
        schema=SCHEMA,
        func=func,
    )

### IP Geolocator Utility Function

 - Uses an IP address to geolocate a location on Earth.

In [2]:
def geolocate_ip(ip: str) -> str:
    """
    Returns a IP geolocation information of the given ip address.

    Args:
        ip: The ip address to geolocate

    Returns:
        JSON which represents the geolocation of the given ip.
    """
    import requests

    return requests.get("http://ip-api.com/json/" + ip).json()


register(geolocate_ip)

### GitHub Repository Search Utility Function

 - Searches GitHub repositories specified by the given keywords.

In [3]:
def search_github(keywords: str) -> str:
    """
    Searches the GitHub API for repositories using an input keyword.

    Args:
        keywords: The space separated search keywords.

    Returns:
        A JSON of GitHub repositories returned for the keyword list.
        Each result key/value consists of the repository name and url.
    """
    import json
    import requests

    return requests.get(
        f"https://api.github.com/search/repositories?q={'+'.join(keywords.split())}&sort=stars&order=desc"
    ).json()


register(search_github)